In [1]:
import pandas as pd

data = {
    'movie_id': [1, 2, 3],
    'title': ['The Matrix', 'The Dark Knight', 'Interstellar'],
    'overview': [
        'A computer hacker learns about the true nature of his reality and his role in the war against its controllers.',
        'Batman must accept one of the greatest psychological and physical tests when the Joker wreaks havoc.',
        'A team of explorers travel through a wormhole in space in an attempt to ensure humanity\'s survival.'
    ],
    'genres': ['Action|Sci-Fi', 'Action|Crime|Drama', 'Adventure|Drama|Sci-Fi'],
    'keywords': ['reality|artificial', 'joker|gotham', 'space|wormhole'],
    'cast': ['Keanu Reeves|Laurence Fishburne', 'Christian Bale|Heath Ledger', 'Matthew McConaughey|Anne Hathaway'],
    'crew': ['Lana Wachowski|Director', 'Christopher Nolan|Director', 'Christopher Nolan|Director']
}

movies = pd.DataFrame(data)
movies.to_csv('artifacts/movies.csv', index=False)
movies.head()


OSError: Cannot save file into a non-existent directory: 'artifacts'

In [2]:
import os
os.makedirs("artifacts", exist_ok=True)


In [3]:
import pandas as pd
import os

In [4]:
data = {
    'movie_id': [1, 2, 3],
    'title': ['The Matrix', 'The Dark Knight', 'Interstellar'],
    'genres': ['Action|Sci-Fi', 'Action|Crime|Drama', 'Adventure|Drama|Sci-Fi'],
    'overview': [
        'A computer hacker learns from mysterious rebels...',
        'When the menace known as the Joker emerges...',
        'A team of explorers travel through a wormhole...'
    ],
    'keywords': ['hacker|rebel|matrix', 'joker|chaos|batman', 'space|wormhole|time'],
    'cast': ['Keanu Reeves|Laurence Fishburne', 'Christian Bale|Heath Ledger', 'Matthew McConaughey|Anne Hathaway'],
    'crew': ['Lana Wachowski|Director', 'Christopher Nolan|Director', 'Christopher Nolan|Director']
}

In [5]:
movies = pd.DataFrame(data)

In [6]:
os.makedirs("artifacts", exist_ok=True)

In [8]:
movies.to_csv('artifacts/movies.csv', index=False)

In [9]:
import pandas as pd

In [10]:
movies = pd.read_csv('artifacts/movies.csv')

In [11]:
def collapse_features(row):
    return ' '.join([
        row['overview'],
        row['keywords'].replace('|', ' '),
        row['cast'].replace('|', ' '),
        row['crew'].replace('|', ' ')
    ])

In [12]:
movies['tags'] = movies.apply(collapse_features, axis=1)

In [14]:
movies = movies[['movie_id', 'title', 'tags']]

In [15]:
movies.head()

,movie_id,title,tags
0,1,The Matrix,A computer hacker learns from mysterious rebel...
1,2,The Dark Knight,When the menace known as the Joker emerges... ...
2,3,Interstellar,A team of explorers travel through a wormhole....


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the vectorizer
tfidf = TfidfVectorizer(stop_words='english')

# Replace NaN with empty string (just in case)
movies['tags'] = movies['tags'].fillna('')

# Fit and transform the tags
tfidf_matrix = tfidf.fit_transform(movies['tags'])

# Check the shape of the resulting matrix
tfidf_matrix.shape


(3, 36)

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity between all movie vectors
similarity = cosine_similarity(tfidf_matrix)

# Check the shape of the similarity matrix
similarity.shape


(3, 3)

In [19]:
def recommend(movie):
    movie = movie.lower()
    if movie not in movies['title'].str.lower().values:
        return "Movie not found in dataset."

    index = movies[movies['title'].str.lower() == movie].index[0]
    distances = similarity[index]
    
    # Get top 2 similar movie indexes (excluding the input movie itself)
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:3]

    for i in movie_list:
        print(movies.iloc[i[0]].title)


In [20]:
recommend("The Matrix")


Interstellar
The Dark Knight
